In [ ]:
using Revise

In [ ]:
using RigidBodyDynamics
using RigidBodyDynamics.PDControl
const rbd = RigidBodyDynamics
using MeshCatMechanisms
using MeshCat
using Statistics
using LightXML
using StaticArrays
using Optim
using LinearAlgebra
using RigidBodySim

In [ ]:
vis = Visualizer()

In [ ]:
open(vis)

In [ ]:
"""
Parse the Drake-specific `loop_joint` tags from a URDF and use them
to add non-tree joints to the given mechanism.
"""
function add_loop_joints!(mechanism::Mechanism{T}, urdf::AbstractString) where T
    doc = parse_file(urdf)
    xml_root = LightXML.root(doc)
    xml_loops = get_elements_by_tagname(xml_root, "loop_joint")
    for xml_loop in xml_loops
        name = attribute(xml_loop, "name")
        @assert attribute(xml_loop, "type") == "continuous"
        axis = SVector{3}(rbd.parse_vector(T, find_element(xml_loop, "axis"), "xyz", "1 0 0"))
        joint = Joint(name, Revolute(axis))
        xml_link1 = find_element(xml_loop, "link1")
        body1 = findbody(mechanism, attribute(xml_link1, "link"))
        H1 = Transform3D(frame_before(joint), default_frame(body1),
            rbd.parse_pose(T, xml_link1)...)
        xml_link2 = find_element(xml_loop, "link2")
        body2 = findbody(mechanism, attribute(xml_link2, "link"))
        H2 = Transform3D(frame_after(joint), default_frame(body2),
            rbd.parse_pose(T, xml_link2)...)
        attach!(mechanism, body1, body2, joint,
            joint_pose = H1,
            successor_pose = inv(H2))
    end
end         

In [ ]:
"""
Return a function which maps a configuration vector
for the given mechanism to the total squared error in
the alignment of the robot's non-tree joints.
"""
function loop_joint_error(robot)
    let statecache = StateCache(robot), resultcache = DynamicsResultCache(robot)
        function(q)
            state = statecache[eltype(q)]
            result = resultcache[eltype(q)]
            set_configuration!(state, q)
            zero_velocity!(state)
            gains = rbd.CustomCollections.ConstDict{JointID}(SE3PDGains(PDGains(1, 0), PDGains(1, 0)))
            rbd.constraint_bias!(result, state; stabilization_gains=gains)
            ret = result.constraintbias ⋅ result.constraintbias
        end
    end
end

In [ ]:
urdf = "data/Strandbeest.urdf"
robot = parse_urdf(Float64, urdf)
add_loop_joints!(robot, urdf)
mvis = MechanismVisualizer(robot, URDFVisuals(urdf), vis)

In [ ]:
# Find an initial state of the robot which correctly aligns all 
# the loop joints.

# First, initialize the state with a configuration within the 
# joint limits:
bounds = collect(Iterators.flatten(rbd.position_bounds.(tree_joints(robot))))
lb = rbd.lower.(bounds)
ub = rbd.upper.(bounds);
state = MechanismState(robot)
set_configuration!(state, clamp.(configuration(state), lb, ub))

# Use Optim's Fminbox to minimize the loop joint error within
# the joint limits:
cost = loop_joint_error(robot)
result = Optim.optimize(cost, lb, ub, 
    Vector(configuration(state)), 
    Fminbox(LBFGS()), autodiff=:forward)

# Verify that we've actually closed all the loops
@assert Optim.minimum(result) < 1e-9

# Visualize the result
set_configuration!(state, Optim.minimizer(result))
copyto!(mvis, state)

In [ ]:
# Simulate the robot

# Our controller will try to drive the central crank shaft
# to a constant speed of 1 radian per second:
controller = function (τ, t, state)
    any(isnan, configuration(state)) && error("NaN at time: $t")
    τ .= 0
    τ[2] = -10.0 * (velocity(state)[2] - 1.0)
end

dynamics = Dynamics(robot, controller)
problem = ODEProblem(dynamics, state, (0, 20.0))
@time solution = solve(problem, Tsit5(), abs_tol=1e-6, dt=1e-6)
setanimation!(mvis, solution)